# K-Means

In this notebook you will use GPU-accelerated K-means to find the best locations for a fixed number of humanitarian supply airdrop depots.

## Objectives

By the time you complete this notebook you will be able to:

- Use GPU-accelerated K-means
- Use cuXfilter to visualize K-means clusters

## Imports

For the first time we import `cuml`, the RAPIDS GPU-accelerated library containing many common machine learning algorithms. We will be visualizing the results of your work in this notebook, so we also import `cuxfilter`.

In [1]:
import cudf
import cuml

import cuxfilter as cxf

## Load Data

For this notebook we load again the cleaned UK population data--in this case, we are not specifically looking at counties, so we omit that column and just keep the grid coordinate columns.

In [2]:
gdf = cudf.read_csv('./data/pop_2-03.csv', usecols=['easting', 'northing'])
print(gdf.dtypes)
gdf.shape

northing    float64
easting     float64
dtype: object


(58479894, 2)

In [30]:
gdf

,northing,easting,cluster,dropoff_x,dropoff_y
0,515491.5313,430772.1875,3,-27785.325529,7.124934e+06
1,503572.4688,434685.8750,3,-47756.851857,7.132030e+06
2,517903.6563,432565.5313,3,-23638.264752,7.127867e+06
3,517059.9063,427660.6250,3,-25269.435921,7.119606e+06
4,509228.6875,425527.7813,3,-38561.703330,7.116320e+06
...,...,...,...,...,...
58479889,192393.7813,340519.8438,0,-566583.296940,6.968560e+06
58479890,183451.8906,337749.6250,0,-581159.970068,6.963319e+06
58479891,189997.5156,341165.1250,0,-570591.445808,6.969457e+06
58479892,184439.1250,335227.6563,0,-579343.011249,6.959217e+06


## K-Means Clustering

The unsupervised K-means clustering algorithm will look for a fixed number *k* of centroids in the data and clusters each point with its closest centroid. K-means can be effective when the number of clusters *k* is known or has a good estimate (such as from a model of the underlying mechanics of a problem).

Assume that in addition to knowing the distribution of the population, which we do, we would like to estimate the best locations to build a fixed number of humanitarian supply depots from which we can perform airdrops and reach the population most efficiently. We can use K-means, setting *k* to the number of supply depots available and fitting on the locations of the population, to identify candidate locations.

GPU-accelerated K-means is just as easy as its CPU-only scikit-learn counterpart. In this series of exercises, you will use it to optimize the locations for 5 supply depots.

## Exercise: Make a `KMeans` Instance for 5 Clusters

`cuml.KMeans()` will initialize a K-means instance. Use it now to initialize a K-means instance called `km`, passing the named argument `n_clusters` set equal to our desired number `5`:

#### Solution

In [5]:
# %load solutions/make_k-means_instance
km = cuml.KMeans(n_clusters=5)


## Exercise: Fit to Population

Use the `km.fit` method to fit `km` to the population's locations by passing it the population data. After fitting, add the cluster labels back to the `gdf` in a new column named `cluster`. Finally, you can use `km.cluster_centers_` to see where the algorithm created the 5 centroids.

#### Solution

In [7]:
# %load solutions/km_fit
km.fit(gdf)
gdf['cluster'] = km.labels_
km.cluster_centers_


,0,1
0,150988.668019,315543.991697
1,294886.581587,439240.631466
2,402934.342187,395973.013652
3,540894.692061,415736.503333
4,180242.130118,530767.081392


In [10]:
gdf = gdf[gdf['cluster']!=1]

In [79]:
from pyproj import Transformer


trans = Transformer.from_crs('epsg:27700', 'epsg:3857')
gdf['x'], gdf['y'] = trans.transform(gdf['easting'].values_host, gdf['northing'].values_host)

## Visualize the Clusters

To help us understand where clusters are located, we make a visualization that separates them, using the same three steps as before.

### Associate a Data Source with cuXfilter

In [44]:
cxf_data = cxf.DataFrame.from_dataframe(gdf)

### Define Charts and Widgets

In this case, we have an existing integer column to use with multi-select: `cluster`. We use the same technique to scale the scatterplot, then add a widget to let us select which cluster to look at.

In [91]:
import seaborn as sns


chart_width = 600
palet = sns.color_palette("pastel")
scatter_chart = cxf.charts.datashader.scatter(x='x', y='y', 
                                              width=chart_width, tile_provider="CartoLight", 
                                              aggregate_col='cluster', aggregate_fn='max',
                                              height=int((gdf['northing'].max() - gdf['northing'].min()) / 
                                                         (gdf['easting'].max() - gdf['easting'].min()) *
                                                          chart_width))
# color_palette = palet # НЕ ПОНЯЛ НИЧЕГО ЧЕ НЕ РАБОТАЕТ 
cluster_widget = cxf.charts.panel_widgets.multi_select('cluster')

### Create and Show the Dashboard

In [92]:
dash = cxf_data.dashboard(charts=[scatter_chart],sidebar=[cluster_widget], theme=cxf.themes.dark, data_size_widget=True)

In [93]:
scatter_chart.view()

Card(sizing_mode='scale_width', title='Scatter plot f..., width=600)
    [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')

In [ ]:
%%js
var host = window.location.host;
element.innerText = "'http://"+host+"'";

Set `my_url` in the next cell to the value just printed, making sure to include the quotes and ignoring the button (due to this contained cloud environment) as before:

In [ ]:
my_url = # TODO: Set this value to the print out of the cell above, including the quotes.
dash.show(my_url, port=8789)

... and you can run the next cell to generate a link to the dashboard:

In [ ]:
%%js
var host = window.location.host;
var url = 'http://'+host+'/lab/proxy/8789/';
element.innerHTML = '<a style="color:blue;" target="_blank" href='+url+'>Open Dashboard</a>';

In [ ]:
dash.stop()

<br>
<div align="center"><h2>Please Restart the Kernel</h2></div>

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Next

In the next notebook, you will use GPU-accelerated DBSCAN to identify geographically dense clusters of infected people.